In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import pennylane as qml
from matplotlib import pyplot as plt
from pennylane import numpy as np
import scipy
import networkx as nx
import copy

In [78]:
qml.enable_tape()

In [79]:
weighted_graph_adj_matrix = np.array([
    [0., 1., 2., 1.],
    [1., 0., 3., 2.],
    [2., 3., 0., 1.],
    [1., 2., 1., 0.]
])

In [80]:
from qgscnn import CouplingGate


In [81]:
def compute_L_matrix(adj_matrix):
    vertex_sums = np.sum(adj_matrix, axis=0)
    out = np.zeros(adj_matrix.shape)
    for j in range(len(adj_matrix)):
        for k in range(len(adj_matrix)):
            if j != k:
                out[j][k] = -1. * adj_matrix[j][k]
            else:
                out[j][k] = vertex_sums[j] - adj_matrix[j][k]
                
    return out
        

In [82]:
def target_hamiltonian(L_matrix):
    out = np.zeros((2**len(L_matrix),2**len(L_matrix)))
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            if i == j or L_matrix[i][j]==0.:
                pass
            else:
                m = 1
                for k in range(len(L_matrix)):
                    if k == i or k == j:
                        outer_prod = np.array([
                            [0.,0.],
                            [0., 1.]
                        ])

                        m = np.kron(m, outer_prod)
                    else:
                        m = np.kron(m, np.identity(2))
                               
                out += L_matrix[i][j] * m
            
    return out + np.identity(2**len(L_matrix))

In [83]:
def target_to_qml_hamiltonian(target):
    """
    Helper function to turn a matrix representing the target Hamiltonian into a Pennylane Hamiltonian object
    """
    pauli_coeffs, obs_list = qml.utils.decompose_hamiltonian(target)
    return qml.Hamiltonian(pauli_coeffs, obs_list)
    

In [84]:
def coupling_sublayer(L_matrix, weights, wires):
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            outer_prod_i = np.array([
                [1.-np.exp(-1j*weights[j]/2),0.],
                [0., 1.-np.exp(-1j*weights[j]/2)]
            ])

            outer_prod_j = np.array([
                [1.-np.exp(-1j*weights[j]/2), 0.],
                [0., 1.+np.exp(-1j*weights[j]/2)]
            ])

            gate_matrix = L_matrix[i][j] * np.kron(outer_prod_i, outer_prod_j)

            CouplingGate(gate_matrix, weights[i], weights[j], wires=list(set([i,j])))
    

In [85]:
from pennylane.templates.subroutines import ArbitraryUnitary

In [106]:
def qgscnn_layer(weights, wires, L_matrix):
    # add the coupling gates
    weight_counter = 0
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            if i != j:
                if L_matrix[i][j] != 0.:
                    ArbitraryUnitary(weights[len(L_matrix)+weight_counter*15:len(L_matrix)+(weight_counter+1)*15], wires=[i,j])
                    
    print("HI")

    # add X rotations for the kinetic hamiltonian
    for i in range(len(L_matrix)):
        qml.RX(weights[:len(L_matrix)], wires=i)
    

In [107]:
def quantum_clustering(adj_matrix):

    # the quantum device
    dev = qml.device("default.qubit", wires=len(adj_matrix))
    
    # compute the L matrix
    L_matrix = compute_L_matrix(adj_matrix)
    
    # helps the cost fn if we declare how many vertices graph has
    num_vertices = len(adj_matrix)
    
    # compute the target Hamiltonian
    target = target_hamiltonian(L_matrix)
    
    target_H = target_to_qml_hamiltonian(target)
    
    # model set up
    optimiser = qml.AdamOptimizer(stepsize=0.5)
    steps = 100
    num_layers = 5
    
    def qgscnn(weights, wires, L_matrix):
        for i in range(num_layers):
            qgscnn_layer(weights[i], wires, L_matrix)
    
    # Defines the new QNode
    qnode = qml.QNode(qgscnn, dev)

    def ansatz(params, **kwargs):
        return qgscnn(params, wires=range(len(adj_matrix)), L_matrix=L_matrix)
        

    cost_fn = qml.ExpvalCost(ansatz, target_H, dev)
    
    # to make life easier, kinetic weights are weights[layer][1], and coupling weights are weights[layer][0]
    weights = np.random.randint(-20, 20, size=(num_layers, (15*np.count_nonzero(adj_matrix!=0)+len(adj_matrix))))/50

        
    init = copy.copy(weights)

    # Executes the optimization method
    iterations = 0

    for i in range(0, steps):
        print(f"Iteration {i}")
        weights = optimiser.step(cost_fn, weights)

    return weights, init

In [108]:
weights, init = quantum_clustering(weighted_graph_adj_matrix)

Iteration 0
HI
HI
HI
HI
HI


ValueError: operands could not be broadcast together with shapes (4,) (2,2) 

In [43]:
def qgscnn_state(weights, wires, L_matrix):
        for i in range(num_layers):
            qgscnn_layer(weights[i], wires, L_matrix)
            
        return qml.state()

In [44]:
# compute the L matrix
L_matrix = compute_L_matrix(weighted_graph_adj_matrix)
num_layers = 5

In [52]:
def qgscnn(weights, wires, L_matrix):
    for i in range(num_layers):
        qgscnn_layer(weights[i], wires, L_matrix)

In [53]:
# the quantum device
dev = qml.device("default.qubit", wires=len(weighted_graph_adj_matrix))

@qml.qnode(dev)
def predict(weights):
    wires = range(len(weighted_graph_adj_matrix))
    qgscnn(weights, wires, L_matrix)
    return qml.state()

In [54]:
state = predict(weights)

In [57]:
# compute the target Hamiltonian
target = target_hamiltonian(L_matrix)

target_H = target_to_qml_hamiltonian(target)

In [59]:
eig_v = np.matmul(target, state)

In [61]:
eig_v / state

tensor([  1.-0.00000000e+00j,   1.-6.60286174e-19j,   1.-0.00000000e+00j,
         -1.-0.00000000e+00j,   1.+1.67169586e-19j,  -3.-8.69348542e-19j,
         -5.+5.89890842e-18j, -11.+1.72146211e-23j,   1.+1.64248547e-19j,
         -1.-0.00000000e+00j,  -3.-0.00000000e+00j,  -7.-0.00000000e+00j,
         -1.-0.00000000e+00j,  -7.-0.00000000e+00j, -11.-6.92048174e-23j,
        -19.+0.00000000e+00j], requires_grad=True)